# Analysing a world energy CSV with `pandas`

In this tutorial we analyse a small dataset with information on countries in the world:

- primary energy use per person (GJ per capita)
- GDP per capita (in USD)
- population size (in millions)
- CO₂ emissions per person (tons per capita)

We will:

1. Load the data from a CSV into a `pandas` DataFrame.
2. Get a quick overview of the dataset.
3. Slice the DataFrame into **high-income** and **low-income** groups.
4. Compute summary statistics: **mean**, **median**, **mode**, **standard deviation**.
5. Visualise the data using histograms and simple bar charts.
6. Create new columns such as **energy per unit of GDP** and **emissions per unit of energy**.
7. Give a very short outlook on **regressions** and **distributions**.

Later, you can replace the mock CSV with a full real-world dataset.


## 1. Setup: import libraries

We start by importing `pandas` and `matplotlib`. `matplotlib` is the standard plotting library that works nicely inside Jupyter.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pd.__version__

## 2. Loading the CSV file

In this tutorial we work with a **small mock CSV file** called `world_energy_mock.csv`.

In a real project you would:

- place a CSV file in your repository (for example: `data/world_energy_full.csv`)
- read it with `pd.read_csv("data/world_energy_full.csv")`.

Below, the first cell shows how you would usually load a CSV. The second cell creates a mock CSV if it does not exist, so the notebook is fully self-contained.


In [ ]:
# Usual pattern in your own project:
# df = pd.read_csv("data/world_energy_full.csv")

# For this tutorial we use a small mock file.
# If you have your own CSV, replace the file name here.
csv_path = "world_energy_mock.csv"

# The next cell will create this file if it does not exist.


In [ ]:
# Create a small mock CSV if it is not present in the current folder.
# (You can delete this cell when you work with a real dataset.)

import os

if not os.path.exists(csv_path):
    mock_data = pd.DataFrame({
        "country": ["Switzerland", "Austria", "Germany", "India", "United States", "Brazil", "Nigeria", "China"],
        "primary_energy_GJ_per_capita": [160.0, 170.0, 190.0, 30.0, 320.0, 90.0, 25.0, 120.0],
        "gdp_per_capita_USD": [90000, 60000, 55000, 2500, 75000, 15000, 2000, 12000],
        "population_million": [8.8, 9.0, 84.0, 1400.0, 333.0, 214.0, 220.0, 1410.0],
        "co2_tons_per_capita": [4.5, 5.0, 7.5, 2.0, 14.0, 2.5, 0.7, 7.0]
    })
    mock_data.to_csv(csv_path, index=False)

df = pd.read_csv(csv_path)
df.head()

## 3. Getting a quick overview

A few very useful first steps when exploring any new dataset:

- `df.shape` – how many rows and columns?
- `df.columns` – what are the column names?
- `df.info()` – what are the data types? Any missing values?
- `df.describe()` – summary statistics for numerical columns.


In [ ]:
# Number of rows and columns
df.shape

In [ ]:
# Column names
df.columns

In [ ]:
# Data types and missing values
df.info()

In [ ]:
# Basic descriptive statistics for numerical columns
df.describe()

## 4. Slicing the DataFrame: high-income and low-income countries

We now create simple groups based on **GDP per capita**. These are not official categories, just illustrative thresholds:

- **High-income**: GDP per capita > 50,000 USD  
- **Low-income**: GDP per capita < 10,000 USD

We first keep the original DataFrame with a country index for easier reading.


In [ ]:
# Use country names as index
df = df.set_index("country")
df

In [ ]:
# Define boolean masks
high_income_mask = df["gdp_per_capita_USD"] > 50000
low_income_mask = df["gdp_per_capita_USD"] < 10000

# Apply masks to create slices
df_high_income = df[high_income_mask]
df_low_income = df[low_income_mask]

df_high_income

In [ ]:
df_low_income

We can also slice based on **energy use** or **emissions**. For example, countries with very high primary energy use (more than 150 GJ per capita):

In [ ]:
high_energy_mask = df["primary_energy_GJ_per_capita"] > 150
df_high_energy = df[high_energy_mask]
df_high_energy

And we can combine conditions with the logical operators `&` (and), `|` (or), `~` (not). For instance: countries that are high-income **and** high-energy:

In [ ]:
df_high_income_high_energy = df[high_income_mask & high_energy_mask]
df_high_income_high_energy

## 5. Summary statistics: mean, median, mode, standard deviation

We now look at classical summary statistics for a single column, for example `primary_energy_GJ_per_capita`.

### 5.1 Whole sample


In [ ]:
energy = df["primary_energy_GJ_per_capita"]

mean_energy = energy.mean()
median_energy = energy.median()
mode_energy = energy.mode()      # can return multiple values
std_energy = energy.std()

mean_energy, median_energy, mode_energy.tolist(), std_energy

In this continuous setting, the **mode** is often not very informative because many countries may have distinct values. It is more useful when there are repeated or discrete values (e.g. binned income classes).

We can also compute several statistics at once for multiple columns using `DataFrame.agg`:


In [ ]:
columns_of_interest = ["primary_energy_GJ_per_capita",
                         "gdp_per_capita_USD",
                         "co2_tons_per_capita"]

df[columns_of_interest].agg(["mean", "median", "std"])

### 5.2 High-income vs low-income

Now we compare the same statistics between high-income and low-income groups.

In [ ]:
df_high_income[columns_of_interest].agg(["mean", "median", "std"])

In [ ]:
df_low_income[columns_of_interest].agg(["mean", "median", "std"])

## 6. Visualising distributions

We now create a few simple plots using `matplotlib`:

- histograms for primary energy use, GDP per capita, and emissions
- a small bar chart comparing mean values


In [ ]:
# Histogram for primary energy use
plt.figure()
df["primary_energy_GJ_per_capita"].hist(bins=8)
plt.xlabel("Primary energy (GJ per capita)")
plt.ylabel("Number of countries")
plt.title("Distribution of primary energy use")

### 📝 Exercise: make your own histogram

Create a histogram for **GDP per capita** using the column `gdp_per_capita_USD`.

Try to:

- choose a reasonable number of bins (for example 8 or 10),
- label the x-axis and y-axis,
- add a short title.

Once you have tried it yourself, you can reveal the solution below (in a hidden code cell).

In [ ]:
# Histogram for GDP per capita
plt.figure()
df["gdp_per_capita_USD"].hist(bins=8)
plt.xlabel("GDP per capita (USD)")
plt.ylabel("Number of countries")
plt.title("Distribution of GDP per capita")

In [ ]:
# Histogram for CO2 emissions per capita
plt.figure()
df["co2_tons_per_capita"].hist(bins=8)
plt.xlabel("CO2 emissions (tons per capita)")
plt.ylabel("Number of countries")
plt.title("Distribution of CO2 emissions per capita")

### 6.1 Bar chart of mean values

We can also compare mean values across variables in a simple bar chart.


In [ ]:
means = df[columns_of_interest].mean()

plt.figure()
plt.bar(means.index, means.values)
plt.ylabel("Mean value")
plt.title("Mean of selected indicators")
plt.xticks(rotation=45);

## 7. Creating new ecological-economic indicators

We now add some derived variables that are often interesting in ecological economics.

1. **Energy intensity of GDP** (energy per 1,000 USD of GDP):
   \[
   \text{energy\_per\_1000usd} = \frac{\text{primary\_energy\_GJ\_per\_capita}}{\text{gdp\_per\_capita\_USD} / 1000}.
   \]

2. **Emissions per unit of energy**:
   \[
   \text{co2\_per\_GJ} = \frac{\text{CO₂ tons per capita}}{\text{primary\_energy\_GJ\_per\_capita}}.
   \]

3. **Total primary energy use per country** (in GJ):
   \[
   \text{total\_energy\_GJ} = \text{primary\_energy\_GJ\_per\_capita} \times \text{population\_million} \times 10^6.
   \]


In [ ]:
df["energy_per_1000usd"] = df["primary_energy_GJ_per_capita"] / (df["gdp_per_capita_USD"] / 1000)
df["co2_per_GJ"] = df["co2_tons_per_capita"] / df["primary_energy_GJ_per_capita"]
df["total_energy_GJ"] = df["primary_energy_GJ_per_capita"] * df["population_million"] * 1e6

df[["primary_energy_GJ_per_capita", "gdp_per_capita_USD",
    "energy_per_1000usd", "co2_per_GJ", "total_energy_GJ"]].head()

We can also look at the distribution of these new indicators.

In [ ]:
plt.figure()
df["energy_per_1000usd"].hist(bins=8)
plt.xlabel("Energy per 1000 USD of GDP (GJ)")
plt.ylabel("Number of countries")
plt.title("Energy intensity of GDP")

In [ ]:
plt.figure()
df["co2_per_GJ"].hist(bins=8)
plt.xlabel("CO2 per GJ of energy (tons/GJ)")
plt.ylabel("Number of countries")
plt.title("Emissions intensity of energy")

## 8. Outlook: regressions and distributions

So far we have only looked at **descriptive statistics**. Often we want to know whether there is a
**systematic relationship** between two variables. For example:

- Do countries with higher GDP per capita also use more energy?
- Is higher energy use associated with higher CO₂ emissions?

A very common tool for this is a **regression model**, such as a simple linear regression:

\[
\text{energy} = a + b \cdot \text{GDP} + \varepsilon,
\]

where

- \(a\) is an intercept,
- \(b\) is a slope (how much energy increases when GDP increases),
- \(\varepsilon\) is an error term (everything we do not explain).

We will study regressions more systematically later. For now we can at least look at a **scatter plot**
to see whether there is a clear pattern.


In [ ]:
plt.figure()
plt.scatter(df["gdp_per_capita_USD"], df["primary_energy_GJ_per_capita"])
plt.xlabel("GDP per capita (USD)")
plt.ylabel("Primary energy (GJ per capita)")
plt.title("GDP vs primary energy use")


### Distributions

Many real-world variables in ecological economics are **not** normally distributed. For example:

- income and wealth often have **heavy tails** (a few very high values),
- city sizes and firm sizes can follow approximate **power-law** or **log-normal** distributions,
- energy use per capita is often very skewed across countries.

Looking at histograms and summary statistics is a first step towards understanding these distributions.
Later in the course we will:

- study distribution shapes more carefully,
- connect them to underlying mechanisms (e.g. multiplicative growth, network effects),
- and use statistical tools to fit and compare different distributions.


## 9. Mini-exercises

Try the following small tasks to practice:

1. **Change the thresholds** for "high-income" and "low-income" and see how your slices change.
2. Compute the **mean CO₂ emissions per capita** for high-income vs low-income countries. Which group emits more?
3. Create a new indicator `co2_per_1000usd` (CO₂ per 1,000 USD of GDP) and compare its mean across groups.
4. Make a scatter plot of `energy_per_1000usd` (x-axis) and `co2_per_GJ` (y-axis). Do countries with more energy-intensive GDP also have more carbon-intensive energy?

Once you are comfortable with this small mock dataset, you can plug in a **real global dataset** and repeat the same analysis.


In [ ]:
# Solutions for Section 9 mini-exercises
# (Hidden in the book via the 'hide-cell' tag.)

# 1. Change thresholds for "high-income" and "low-income"
# Example: high-income > 40,000 USD, low-income < 8,000 USD
high_income_mask_2 = df["gdp_per_capita_USD"] > 40000
low_income_mask_2 = df["gdp_per_capita_USD"] < 8000

df_high_income_2 = df[high_income_mask_2]
df_low_income_2 = df[low_income_mask_2]

# 2. Mean CO2 emissions per capita for high-income vs low-income countries
mean_co2_high = df_high_income["co2_tons_per_capita"].mean()
mean_co2_low = df_low_income["co2_tons_per_capita"].mean()

print("Mean CO2 per capita (high-income):", mean_co2_high)
print("Mean CO2 per capita (low-income):", mean_co2_low)

# 3. New indicator: CO2 per 1000 USD of GDP
df["co2_per_1000usd"] = df["co2_tons_per_capita"] / (df["gdp_per_capita_USD"] / 1000)

co2_indicator = df.groupby(high_income_mask)["co2_per_1000usd"].mean()
print("\nMean CO2 per 1000 USD of GDP")
print("High-income countries:", co2_indicator[True])
print("Other countries:", co2_indicator[False])

# 4. Scatter plot: energy_per_1000usd (x-axis) vs co2_per_GJ (y-axis)
plt.figure()
plt.scatter(df["energy_per_1000usd"], df["co2_per_GJ"])
plt.xlabel("Energy per 1000 USD of GDP (GJ)")
plt.ylabel("CO2 per GJ of energy (tons/GJ)")
plt.title("Energy intensity vs emissions intensity")
